In [1]:
from sqlalchemy import create_engine
from sqlalchemy import text

# Core Engine and Connection objects

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [3]:
# The transaction is not committed automatically
# It executes BEGIN implicitly and ROLLBACK at the end
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2024-08-11 16:02:58,260 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,262 INFO sqlalchemy.engine.Engine select 'hello world'
2024-08-11 16:02:58,263 INFO sqlalchemy.engine.Engine [generated in 0.00279s] ()
[('hello world',)]
2024-08-11 16:02:58,264 INFO sqlalchemy.engine.Engine ROLLBACK


In [4]:
# Style: Commit as you go
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2024-08-11 16:02:58,279 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,280 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2024-08-11 16:02:58,281 INFO sqlalchemy.engine.Engine [generated in 0.00250s] ()
2024-08-11 16:02:58,283 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-08-11 16:02:58,284 INFO sqlalchemy.engine.Engine [generated in 0.00153s] [(1, 1), (2, 4)]
2024-08-11 16:02:58,286 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
# Style: Begin once
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2024-08-11 16:02:58,300 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,301 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-08-11 16:02:58,302 INFO sqlalchemy.engine.Engine [cached since 0.01964s ago] [(6, 8), (9, 10)]
2024-08-11 16:02:58,303 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
# The result object represents an iterable object of result rows
# result has lots of methods for fetching and transforming rows, it also implements
# the Python iterator interface so that we can iterate over the collection of Row objects directly
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    # The Row objects themselves are intended to act like Python named tuples
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-08-11 16:02:58,320 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,321 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-08-11 16:02:58,322 INFO sqlalchemy.engine.Engine [generated in 0.00234s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2024-08-11 16:02:58,323 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
# Another way to get the result is by Mapping Access
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))

    for dict_row in result.mappings():
        x = dict_row["x"]
        y = dict_row["y"]

2024-08-11 16:02:58,338 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,339 INFO sqlalchemy.engine.Engine select x, y from some_table
2024-08-11 16:02:58,340 INFO sqlalchemy.engine.Engine [generated in 0.00239s] ()
2024-08-11 16:02:58,342 INFO sqlalchemy.engine.Engine ROLLBACK


# Passing Parameters

When using textual SQL, a Python literal value, even non-strings like integers or dates, should never be stringified into SQL string directly; a parameter should always be used. This is most famously known as how to avoid SQL injection attacks when the data is untrusted. However it also allows the SQLAlchemy dialects and/or DBAPI to correctly handle the incoming input for the backend

In [8]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-08-11 16:02:58,358 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,360 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2024-08-11 16:02:58,361 INFO sqlalchemy.engine.Engine [generated in 0.00225s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2024-08-11 16:02:58,362 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
# Sending multiple parameters
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2024-08-11 16:02:58,376 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,377 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-08-11 16:02:58,379 INFO sqlalchemy.engine.Engine [cached since 0.09588s ago] [(11, 12), (13, 14)]
2024-08-11 16:02:58,379 INFO sqlalchemy.engine.Engine COMMIT


# Session
The fundamental transactional / database interactive object when using the ORM is called the Session. In modern SQLAlchemy, this object is used in a manner very similar to that of the Connection, and in fact as the Session is used, it refers to a Connection internally which it uses to emit SQL.


**Tip:**

The Session doesn’t actually hold onto the Connection object after it ends the transaction. It gets a new Connection from the Engine the next time it needs to execute SQL against the database.

In [10]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-08-11 16:02:58,574 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,574 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2024-08-11 16:02:58,574 INFO sqlalchemy.engine.Engine [generated in 0.00105s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2024-08-11 16:02:58,574 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
# Commit as you go behavior
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

2024-08-11 16:02:58,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:02:58,602 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2024-08-11 16:02:58,602 INFO sqlalchemy.engine.Engine [generated in 0.00115s] [(11, 9), (15, 13)]
2024-08-11 16:02:58,604 INFO sqlalchemy.engine.Engine COMMIT
